# **Sistem Augmentasi dan Klasifikasi Suara “Buka/Tutup”**

Proyek ini bertujuan untuk membuat sistem pengenalan perintah suara sederhana dengan dua kategori: “buka” dan “tutup”.
Langkah-langkah utama mencakup konversi format audio, augmentasi data, ekstraksi fitur MFCC, pelatihan model klasifikasi (SVM & Random Forest), evaluasi performa, dan penyimpanan model untuk deployment.

## Imports & Konfigurasi

In [14]:
# === 1️⃣ IMPORTS & KONFIGURASI ===
import os
from pydub import AudioSegment
from pydub.utils import which

# === Path utama ===
BASE_IN = r"C:\Kuliah\SEMESTER_5\PSD\voice_predic\voice_ori"
BASE_OUT = r"C:\Kuliah\SEMESTER_5\PSD\voice_predic\voice_augmented"

# === Pastikan ffmpeg terdaftar ===
os.environ["PATH"] += os.pathsep + r"C:\Kuliah\SEMESTER_5\PSD\voice_predic\ffmpeg-7.1.1-essentials_build\bin"
AudioSegment.converter = r"C:\Kuliah\SEMESTER_5\PSD\voice_predic\ffmpeg-7.1.1-essentials_build\bin\ffmpeg.exe"

print("FFmpeg terdeteksi di:", which("ffmpeg"))
print("Converter path:", AudioSegment.converter)

# Buat folder output (jika belum ada)
os.makedirs(os.path.join(BASE_OUT, "buka"), exist_ok=True)
os.makedirs(os.path.join(BASE_OUT, "tutup"), exist_ok=True)


FFmpeg terdeteksi di: C:\Kuliah\SEMESTER_5\PSD\voice_predic\ffmpeg-7.1.1-essentials_build\bin\ffmpeg.exe
Converter path: C:\Kuliah\SEMESTER_5\PSD\voice_predic\ffmpeg-7.1.1-essentials_build\bin\ffmpeg.exe


Penjelasan

Bagian ini berfungsi untuk:

* Mengimpor pustaka pydub dan os untuk pengolahan audio dan manajemen file.

* Menentukan direktori utama (BASE_IN) tempat file asli disimpan dan direktori keluaran (BASE_OUT) untuk hasil augmentasi.

* Mendaftarkan path ffmpeg agar dapat dikenali oleh pydub sebagai konverter audio.

* Membuat folder buka dan tutup di folder output jika belum ada.

* Menampilkan informasi lokasi ffmpeg yang berhasil terdeteksi.

Tujuan utama langkah ini adalah memastikan sistem siap untuk memproses file audio .m4a menjadi .wav.

## Fungsi Konversi Format Audio

In [15]:
# === 2️⃣ Fungsi Konversi ===
def convert_folder_m4a_to_wav(base_folder, categories=("buka","tutup")):
    for cat in categories:
        folder = os.path.join(base_folder, cat)
        print(f"\n📁 Mengecek folder: {folder}")
        if not os.path.isdir(folder):
            print(f"⚠️ Folder tidak ditemukan: {folder}")
            continue

        for f in sorted(os.listdir(folder)):
            if f.lower().endswith((".m4a", ".mka")):
                src = os.path.join(folder, f)
                dst = os.path.join(folder, os.path.splitext(f)[0] + ".wav")

                print(f"🔄 Konversi: {src}")
                try:
                    audio = AudioSegment.from_file(src)
                    audio.export(dst, format="wav")
                    print(f"✅ Converted: {cat}/{f} → {os.path.basename(dst)}")
                except Exception as e:
                    print(f"❌ Gagal convert {src}: {e}")

convert_folder_m4a_to_wav(BASE_IN)



📁 Mengecek folder: C:\Kuliah\SEMESTER_5\PSD\voice_predic\voice_ori\buka
🔄 Konversi: C:\Kuliah\SEMESTER_5\PSD\voice_predic\voice_ori\buka\buka1.m4a
✅ Converted: buka/buka1.m4a → buka1.wav
🔄 Konversi: C:\Kuliah\SEMESTER_5\PSD\voice_predic\voice_ori\buka\buka10.m4a
✅ Converted: buka/buka10.m4a → buka10.wav
🔄 Konversi: C:\Kuliah\SEMESTER_5\PSD\voice_predic\voice_ori\buka\buka2.m4a
✅ Converted: buka/buka2.m4a → buka2.wav
🔄 Konversi: C:\Kuliah\SEMESTER_5\PSD\voice_predic\voice_ori\buka\buka3.m4a
✅ Converted: buka/buka3.m4a → buka3.wav
🔄 Konversi: C:\Kuliah\SEMESTER_5\PSD\voice_predic\voice_ori\buka\buka4.m4a
✅ Converted: buka/buka4.m4a → buka4.wav
🔄 Konversi: C:\Kuliah\SEMESTER_5\PSD\voice_predic\voice_ori\buka\buka5.m4a
✅ Converted: buka/buka5.m4a → buka5.wav
🔄 Konversi: C:\Kuliah\SEMESTER_5\PSD\voice_predic\voice_ori\buka\buka6.m4a
✅ Converted: buka/buka6.m4a → buka6.wav
🔄 Konversi: C:\Kuliah\SEMESTER_5\PSD\voice_predic\voice_ori\buka\buka7.m4a
✅ Converted: buka/buka7.m4a → buka7.wav
🔄 Ko

Fungsi convert_folder_m4a_to_wav() bertugas mengonversi semua file berformat .m4a atau .mka menjadi .wav agar kompatibel dengan pustaka librosa.
Proses dilakukan secara rekursif pada dua kategori folder (buka dan tutup).
Hasil konversi akan disimpan di folder yang sama dengan nama file yang sama namun berformat .wav.
Jika terjadi kesalahan, sistem menampilkan pesan error tetapi tidak menghentikan seluruh proses.

## Pengujian Konversi Menggunakan subprocess

In [16]:
import subprocess

test_src = r"C:\Kuliah\SEMESTER_5\PSD\voice_predic\voice_ori\buka\buka1.m4a"
test_dst = r"C:\Kuliah\SEMESTER_5\PSD\voice_predic\voice_ori\buka\buka1_test.wav"

cmd = [
    r"C:\Kuliah\SEMESTER_5\PSD\voice_predic\ffmpeg-7.1.1-essentials_build\bin\ffmpeg.exe",
    "-y",
    "-i", test_src,
    "-acodec", "pcm_s16le",
    "-ar", "16000",
    test_dst
]

result = subprocess.run(cmd)
print("Return code:", result.returncode)


Return code: 0


Kode ini digunakan untuk menguji ffmpeg secara manual dengan modul subprocess.
Perintah subprocess.run() menjalankan ffmpeg dari command line untuk mengonversi satu file .m4a menjadi .wav dengan:

* format PCM 16-bit linear (pcm_s16le)

* sample rate 16 kHz.

Jika returncode = 0, berarti proses berhasil dan ffmpeg bekerja dengan benar.

## Augmentasi Suara Otomatis

In [20]:
# === 2️⃣ AUGMENTASI SUARA OTOMATIS ===
import librosa
import soundfile as sf
import numpy as np
import random
import os

# Konfigurasi dasar
BASE_IN = r"C:\Kuliah\SEMESTER_5\PSD\voice_predic\voice_ori"
BASE_OUT = r"C:\Kuliah\SEMESTER_5\PSD\voice_predic\voice_augmented"
TARGET_FILES = 100  # jumlah target file per kategori
SAMPLE_RATE = 16000

os.makedirs(os.path.join(BASE_OUT, "buka"), exist_ok=True)
os.makedirs(os.path.join(BASE_OUT, "tutup"), exist_ok=True)

# Fungsi bantu
def load_audio(file_path):
    try:
        audio, sr = librosa.load(file_path, sr=SAMPLE_RATE)
        return audio, sr
    except Exception as e:
        print(f"⚠️ Error load {file_path}: {e}")
        return None, None

def save_audio(audio, sr, output_path):
    try:
        sf.write(output_path, audio, sr)
    except Exception as e:
        print(f"⚠️ Gagal simpan {output_path}: {e}")

def augment_audio(audio, sr):
    """Generate variasi augmentasi dari 1 file"""
    augmented = []
    
    # 1️⃣ Pitch shift
    for n_steps in [-4, -2, -1, 1, 2, 3, 4]:
        shifted = librosa.effects.pitch_shift(y=audio, sr=sr, n_steps=n_steps)
        augmented.append(shifted)
    
    # 2️⃣ Time stretch
    for rate in [0.8, 0.9, 1.1, 1.2]:
        stretched = librosa.effects.time_stretch(y=audio, rate=rate)
        augmented.append(stretched[:len(audio)])  # potong biar sama panjang
    
    # 3️⃣ Noise injection
    for noise_level in [0.005, 0.01, 0.015]:
        noise = np.random.randn(len(audio)) * noise_level
        noisy = audio + noise
        noisy = noisy / np.max(np.abs(noisy))
        augmented.append(noisy)
    
    # 4️⃣ Volume variation
    for factor in [0.7, 0.9, 1.1, 1.3]:
        vol = np.clip(audio * factor, -1.0, 1.0)
        augmented.append(vol)
    
    # 5️⃣ Random combo (pakai keyword argument semua)
    for _ in range(20):
        temp = audio.copy()
        if random.random() > 0.5:
            temp = librosa.effects.pitch_shift(
                y=temp, sr=sr, n_steps=random.uniform(-3, 3)
            )
        if random.random() > 0.5:
            temp = librosa.effects.time_stretch(
                y=temp, rate=random.uniform(0.8, 1.2)
            )
        if random.random() > 0.4:
            noise = np.random.randn(len(temp)) * random.uniform(0.003, 0.015)
            temp += noise
        temp = np.clip(temp, -1.0, 1.0)
        augmented.append(temp)
    
    return augmented


def augment_category(category):
    in_dir = os.path.join(BASE_IN, category)
    out_dir = os.path.join(BASE_OUT, category)
    files = [f for f in os.listdir(in_dir) if f.lower().endswith(".wav")]
    
    all_aug = []
    print(f"\n🎙️ Proses kategori: {category} ({len(files)} file sumber)")
    
    for file in files:
        path = os.path.join(in_dir, file)
        audio, sr = load_audio(path)
        if audio is None:
            continue
        
        all_aug.append(audio)  # simpan versi original
        aug_list = augment_audio(audio, sr)
        all_aug.extend(aug_list)
    
    # Acak dan ambil sesuai target
    random.shuffle(all_aug)
    selected = all_aug[:TARGET_FILES]
    
    print(f"💾 Menyimpan {len(selected)} hasil augmentasi ke {out_dir}")
    for i, audio in enumerate(selected, 1):
        save_audio(audio, SAMPLE_RATE, os.path.join(out_dir, f"{category}_{i}.wav"))
        if i % 10 == 0:
            print(f"  Progress: {i}/{TARGET_FILES}")
    print(f"✅ {category} selesai ({len(selected)} file disimpan)")

# Jalankan augmentasi
augment_category("buka")
augment_category("tutup")



🎙️ Proses kategori: buka (11 file sumber)
💾 Menyimpan 100 hasil augmentasi ke C:\Kuliah\SEMESTER_5\PSD\voice_predic\voice_augmented\buka
  Progress: 10/100
  Progress: 20/100
  Progress: 30/100
  Progress: 40/100
  Progress: 50/100
  Progress: 60/100
  Progress: 70/100
  Progress: 80/100
  Progress: 90/100
  Progress: 100/100
✅ buka selesai (100 file disimpan)

🎙️ Proses kategori: tutup (10 file sumber)
💾 Menyimpan 100 hasil augmentasi ke C:\Kuliah\SEMESTER_5\PSD\voice_predic\voice_augmented\tutup
  Progress: 10/100
  Progress: 20/100
  Progress: 30/100
  Progress: 40/100
  Progress: 50/100
  Progress: 60/100
  Progress: 70/100
  Progress: 80/100
  Progress: 90/100
  Progress: 100/100
✅ tutup selesai (100 file disimpan)


Fungsi utama di sini adalah `augment_audio()` yang menambah variasi suara dari satu file asli menggunakan lima teknik:

1. Pitch Shift — mengubah tinggi nada (misal lebih rendah atau tinggi beberapa semitone).

2. Time Stretch — mempercepat atau memperlambat tempo tanpa mengubah pitch.

3. Noise Injection — menambahkan noise Gaussian ringan untuk simulasi gangguan rekaman.

4. Volume Variation — menambah atau mengurangi volume suara.

5. Random Combination — menggabungkan beberapa augmentasi secara acak.

Fungsi `augment_category()` kemudian menerapkan semua variasi tersebut ke seluruh file dalam kategori (buka dan tutup) dan menyimpan hasilnya hingga mencapai target 100 file per kategori.
Tujuan tahap ini adalah memperbanyak data agar model tidak overfitting terhadap suara asli.

## Ekstraksi Fitur & Pembentukan Dataset

In [27]:
BASE_IN_LIST = [
    r"C:\Kuliah\SEMESTER_5\PSD\voice_predic\voice_augmented",
    r"C:\Kuliah\SEMESTER_5\PSD\voice_predic\voice_ori"
]


In [28]:
# === 3A) Ekstraksi fitur & dataset ===
import os, glob, warnings, json
import numpy as np
import librosa
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import joblib

warnings.filterwarnings("ignore")

# Path dataset augmented
BASE_AUG = r"C:\Kuliah\SEMESTER_5\PSD\voice_predic\voice_augmented"

# Param audio (samakan dengan tahap augmentasi)
TARGET_SR = 16000
FIX_SECONDS = 1.0
FIX_SAMPLES = int(TARGET_SR * FIX_SECONDS)

# --- helper: normalisasi panjang, resample, mono ---
def load_audio_fixed(path, target_sr=TARGET_SR, fix_len=FIX_SAMPLES):
    y, sr = librosa.load(path, sr=target_sr, mono=True)
    if np.max(np.abs(y)) > 0:
        y = y / np.max(np.abs(y))
    if len(y) < fix_len:
        y = np.pad(y, (0, fix_len-len(y)))
    else:
        y = y[:fix_len]
    return y, target_sr

# --- ekstraksi MFCC + delta + delta2, lalu pooling (mean & std) agar jadi vektor fix length ---
def extract_features(y, sr=TARGET_SR, n_mfcc=20, n_fft=512, hop_length=160, win_length=400):
    # mfcc
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc, n_fft=n_fft,
                                hop_length=hop_length, win_length=win_length)
    d1 = librosa.feature.delta(mfcc)
    d2 = librosa.feature.delta(mfcc, order=2)
    feat = np.concatenate([mfcc, d1, d2], axis=0)   # (3*n_mfcc, T)
    # pooling statistik
    mean = np.mean(feat, axis=1)
    std  = np.std(feat, axis=1)
    return np.hstack([mean, std]).astype(np.float32)  # (6*n_mfcc,)

def load_dataset_for_training(base_list=BASE_IN_LIST):
    X, y = [], []
    classes = ["buka", "tutup"]
    for base_aug in base_list:
        for label, cat in enumerate(classes):
            paths = sorted(glob.glob(os.path.join(base_aug, cat, "*.wav")))
            for p in paths:
                ysig, _ = load_audio_fixed(p)
                feats = extract_features(ysig)
                X.append(feats)
                y.append(label)
    return np.array(X), np.array(y), classes


X, y, classes = load_dataset_for_training()
print("X shape:", X.shape, "| y shape:", y.shape, "| kelas:", classes, "| sebaran:", np.bincount(y))


X shape: (221, 120) | y shape: (221,) | kelas: ['buka', 'tutup'] | sebaran: [111 110]


## Pemisahan Data & Pelatihan Model

In [29]:
# === 3B) Train-test split + scaling ===
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42, stratify=y
)

scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_test_s  = scaler.transform(X_test)

# === 3C) Training dua model (SVM RBF & RandomForest) ===
svm_clf = SVC(kernel="rbf", C=5.0, gamma="scale", probability=True, random_state=42)
svm_clf.fit(X_train_s, y_train)

rf_clf = RandomForestClassifier(
    n_estimators=300, max_depth=None, min_samples_split=2, min_samples_leaf=1,
    random_state=42, n_jobs=-1
)
rf_clf.fit(X_train, y_train)  # RF tidak perlu scaling (pakai X asli)


RandomForestClassifier(n_estimators=300, n_jobs=-1, random_state=42)

## Evaluasi Model

In [30]:
def evaluate_model(name, model, X_tr, y_tr, X_te, y_te, use_proba=True):
    y_pred_tr = model.predict(X_tr)
    y_pred_te = model.predict(X_te)

    acc_tr = accuracy_score(y_tr, y_pred_tr)
    acc_te = accuracy_score(y_te, y_pred_te)

    print(f"\n=== {name} ===")
    print(f"Accuracy Train: {acc_tr:.4f}")
    print(f"Accuracy Test : {acc_te:.4f}")
    print("Confusion matrix (Test):")
    print(confusion_matrix(y_te, y_pred_te))
    print("Classification report (Test):")
    print(classification_report(y_te, y_pred_te, target_names=classes))

    # AUC (opsional, untuk binary)
    try:
        if use_proba and len(np.unique(y_te)) == 2:
            proba = model.predict_proba(X_te)[:, 1]
            auc = roc_auc_score(y_te, proba)
            print(f"AUC (binary): {auc:.4f}")
    except Exception:
        pass

# Evaluasi SVM (pakai data yang sudah diskalakan)
evaluate_model("SVM RBF", svm_clf, X_train_s, y_train, X_test_s, y_test, use_proba=True)

# Evaluasi RF (pakai X asli)
evaluate_model("RandomForest", rf_clf, X_train, y_train, X_test, y_test, use_proba=True)



=== SVM RBF ===
Accuracy Train: 1.0000
Accuracy Test : 1.0000
Confusion matrix (Test):
[[23  0]
 [ 0 22]]
Classification report (Test):
              precision    recall  f1-score   support

        buka       1.00      1.00      1.00        23
       tutup       1.00      1.00      1.00        22

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45

AUC (binary): 1.0000

=== RandomForest ===
Accuracy Train: 1.0000
Accuracy Test : 0.9556
Confusion matrix (Test):
[[23  0]
 [ 2 20]]
Classification report (Test):
              precision    recall  f1-score   support

        buka       0.92      1.00      0.96        23
       tutup       1.00      0.91      0.95        22

    accuracy                           0.96        45
   macro avg       0.96      0.95      0.96        45
weighted avg       0.96      0.96      0.96        45

AUC (binary): 0.9980


## Penyimpanan Model

In [32]:
# === 3E) Simpan model terbaik + scaler ===
os.makedirs("model", exist_ok=True)

# pilih yang terbaik berdasar akurasi test
svm_acc = accuracy_score(y_test, svm_clf.predict(X_test_s))
rf_acc  = accuracy_score(y_test, rf_clf.predict(X_test))

if svm_acc >= rf_acc:
    best_name = "svm"
    best_model = svm_clf
    needs_scaler = True
else:
    best_name = "rf"
    best_model = rf_clf
    needs_scaler = True  # kita tetap simpan scaler agar pipeline konsisten (bisa diabaikan saat RF)

joblib.dump(best_model, "model/voice_cmd_best.pkl")
joblib.dump(scaler,     "model/scaler.pkl")
with open("model/classes.json", "w") as f:
    json.dump(classes, f)

print(f"\n✓ Disimpan: model/voice_cmd_best.pkl  (best={best_name}, test_acc={max(svm_acc, rf_acc):.4f})")
print("✓ Disimpan: model/scaler.pkl")
print("✓ Disimpan: model/classes.json")



✓ Disimpan: model/voice_cmd_best.pkl  (best=svm, test_acc=1.0000)
✓ Disimpan: model/scaler.pkl
✓ Disimpan: model/classes.json


## **Deployment**

Deployment akan dilakukan di streamlit. buat beberapa file berikut:


1. app_mic.py
2. requirements.txt
3. masukkan model 
4. masukkan file scaler yang sudah didapatkan tadi



file app_mic.py

```

import streamlit as st
import numpy as np
import librosa
import soundfile as sf
import joblib, json, os, tempfile
from streamlit_mic_recorder import mic_recorder
from sklearn.preprocessing import StandardScaler

# === Konfigurasi ===
MODEL_PATH = "model/voice_cmd_best.pkl"
SCALER_PATH = "model/scaler.pkl"
CLASSES_PATH = "model/classes.json"
TARGET_SR = 16000
FIX_SECONDS = 1.0
FIX_SAMPLES = int(TARGET_SR * FIX_SECONDS)

# === Fungsi bantu ===
@st.cache_resource
def load_assets():
    model = joblib.load(MODEL_PATH)
    scaler = joblib.load(SCALER_PATH)
    classes = json.load(open(CLASSES_PATH))
    return model, scaler, classes

def load_audio_fixed(path, target_sr=TARGET_SR, fix_len=FIX_SAMPLES):
    y, sr = librosa.load(path, sr=target_sr, mono=True)
    if np.max(np.abs(y)) > 0:
        y = y / np.max(np.abs(y))
    if len(y) < fix_len:
        y = np.pad(y, (0, fix_len - len(y)))
    else:
        y = y[:fix_len]
    return y, sr

def extract_features(y, sr=TARGET_SR, n_mfcc=20, n_fft=512, hop_length=160, win_length=400):
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc, n_fft=n_fft,
                                hop_length=hop_length, win_length=win_length)
    d1 = librosa.feature.delta(mfcc)
    d2 = librosa.feature.delta(mfcc, order=2)
    feat = np.concatenate([mfcc, d1, d2], axis=0)
    mean = np.mean(feat, axis=1)
    std = np.std(feat, axis=1)
    return np.hstack([mean, std]).astype(np.float32)

def predict_audio(file_path, model, scaler, classes):
    # pastikan selalu mono, 16kHz, 1 detik
    y, sr = librosa.load(file_path, sr=TARGET_SR, mono=True)

    # hilangkan bagian hening
    y, _ = librosa.effects.trim(y, top_db=30)

    # normalisasi panjang
    if len(y) < FIX_SAMPLES:
        y = np.pad(y, (0, FIX_SAMPLES - len(y)))
    else:
        y = y[:FIX_SAMPLES]

    # normalisasi amplitudo
    if np.max(np.abs(y)) > 0:
        y = y / np.max(np.abs(y))

    # ekstraksi fitur
    feats = extract_features(y, sr).reshape(1, -1)
    feats_scaled = scaler.transform(feats)

    # prediksi
    probs = model.predict_proba(feats_scaled)[0]
    pred = model.predict(feats_scaled)[0]

    label = classes[int(pred)]
    conf = float(np.max(probs))
    return label, conf, dict(zip(classes, probs.tolist()))


# === UI Streamlit ===
st.set_page_config(page_title="🎤 Voice Command Detector (Mic)", layout="centered")
st.title("🎙️ Deteksi Suara 'Buka' / 'Tutup'")
st.markdown("Tekan tombol di bawah untuk merekam suara langsung dari mikrofon.")

# Load model
model, scaler, classes = load_assets()

# === Rekam suara ===
audio_data = mic_recorder(
    start_prompt="🎙️ Tekan untuk mulai merekam",
    stop_prompt="🛑 Tekan lagi untuk berhenti",
    key="recorder",
    just_once=False
)

if audio_data:
    # Simpan hasil rekaman ke file sementara
    with tempfile.NamedTemporaryFile(delete=False, suffix=".wav") as tmp:
        tmp.write(audio_data["bytes"])
        tmp_path = tmp.name

    st.audio(tmp_path, format="audio/wav")
    st.success("✅ Suara berhasil direkam!")

    if st.button("🔍 Prediksi Sekarang"):
        label, conf, probs = predict_audio(tmp_path, model, scaler, classes)

        st.success(f"**Prediksi:** {label.upper()}  \n**Kepercayaan:** {conf*100:.2f}%")
        st.json(probs)

        # Tambahkan threshold kepercayaan
        if conf < 0.7:
            st.warning("⚠️ Suara tidak dikenali dengan cukup yakin. Coba ulangi rekaman.")
        else:
            if label.lower() == "buka":
                st.markdown("🟢 Sistem mengenali suara **BUKA**.")
            elif label.lower() == "tutup":
                st.markdown("🔴 Sistem mengenali suara **TUTUP**.")


```

### Jalankan Deployment

Saya membuat repo baru yang dapat anda akses di: https://github.com/yudhacm/prediksi-suara/blob/main/app_mic.py

kemudian masuk ke streamlit dan hubungkan dengan github anda dan pilih main file di app.py


setelah itu tentukan link url

dan hasil deployment saya dapat dilihat di link berikut:

https://prediksi-suara-230411100057.streamlit.app/